In [31]:
import pandas as pd
import feather
import datetime
import matplotlib.pyplot as plt

# Define the file path
eth_usdt_1m_file = '/allah/freqtrade/user_data/data/binance/futures/ETH_USDT_USDT-1m-futures.feather'

# Read the DataFrame from the feather file
eth_usdt_1m = feather.read_dataframe(eth_usdt_1m_file)
eth_usdt_1m_utc8 = eth_usdt_1m.copy()
# Add 8 hours to the 'date' column and simplify for readability
eth_usdt_1m_utc8['date'] = pd.to_datetime(eth_usdt_1m_utc8['date']) + pd.DateOffset(hours=8)

# Format the 'date' column as a string in the desired format
eth_usdt_1m_utc8['date'] = eth_usdt_1m_utc8['date'].dt.strftime('%Y-%m-%d %H:%M')


In [33]:
eth_usdt_1m_utc8['last_5_volume_sum'] = eth_usdt_1m_utc8['volume'].rolling(window=6).sum() - eth_usdt_1m_utc8['volume']

# Set 'whale_point' to 1 where 'volume' is greater than the rolling sum
eth_usdt_1m_utc8['whale_point'] = (eth_usdt_1m_utc8['volume'] > eth_usdt_1m_utc8['last_5_volume_sum']).astype(int)

In [36]:
eth_usdt_1m_utc8.whale_point.value_counts()

whale_point
0    1992692
1      33583
Name: count, dtype: int64

In [38]:
# average whale_point every month, and plot it 
eth_usdt_1m_utc8['month'] = eth_usdt_1m_utc8['date'].dt.to_period('M')
average_whale_point = eth_usdt_1m_utc8.groupby('month')['whale_point'].mean()

# Plot the average 'whale_point' per month
plt.figure(figsize=(10, 6))
average_whale_point.plot(kind='bar')
plt.xlabel('Month')
plt.ylabel('Average Whale Point')
plt.title('Average Whale Point per Month')
plt.xticks(rotation=45)
plt.show()

AttributeError: Can only use .dt accessor with datetimelike values